# XGBoost Regressor

## Mục tiêu
  - Xây dựng mô hình XGBoost Regressor để dự đoán tuổi thọ trung bình
  - Sử dụng dữ liệu đã được tiền xử lý từ `data/processed/`,
  - Tối ưu hóa siêu tham số bằng 5-Fold Cross-Validation (RandomizedSearchCV)
  - Đánh giá mô hình trên tập train (và validation nếu có): MAE, RMSE, R2
  - Lưu mô hình đã huấn luyện để sử dụng lại (ví dụ cho ensemble)

# Bước 1

### 1.1. Cài đặt thư viện xgboost (nếu cần)

In [1]:
!pip install xgboost --quiet

### 1.2. Import các thư viện cơ bản khác

In [3]:
import os
import joblib
import pandas as pd
import numpy as np
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from scipy.stats import randint, uniform, loguniform
RANDOM_STATE = 42

## Bước 2 - Đọc dữ liệu đã tiền xử lý
Đọc dữ liệu từ `../data/processed/train.csv` và hiển thị thông tin cơ bản.

In [4]:
train_df = pd.read_csv('../data/processed/train.csv')

print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập train: {train_df.shape}")
print("\n5 dòng đầu của tập train:")
train_df.head()

THÔNG TIN DỮ LIỆU
Kích thước tập train: (3255, 15)

5 dòng đầu của tập train:


,country_name,country_code,year,population,poverty_ratio,pop_growth,life_expectancy,gdp_per_capita,gdp_growth,sanitation,electricity,water_access,co2_emissions,slum_population,labor_force
0,Romania,ROU,2019,-0.100387,-0.593216,-1.101137,75.607317,-0.107887,0.079336,1.142611,0.621932,0.746494,-0.047183,-0.857182,-0.871164
1,Mauritius,MUS,2006,-0.235882,-0.685495,-0.500838,72.432195,-0.405891,0.241659,0.204355,0.586683,0.718065,-0.221970,1.767314,-0.189257
2,Angola,AGO,2008,-0.083899,0.283435,1.500859,55.281000,-0.470953,1.327705,-0.839543,-1.545881,-2.268636,-0.432669,0.305989,1.598947
3,Albania,ALB,2001,-0.222240,-0.619770,-1.349940,75.083000,-0.584364,0.930412,-0.835544,0.600783,-0.024271,-0.404910,0.108810,-0.104497
4,Central African Republic,CAF,2021,-0.206911,2.567341,0.236338,40.279000,-0.618434,-0.427187,-1.336901,-2.349558,-3.008624,-0.529029,1.729562,1.172890


In [ ]:
feature_cols = [c for c in train_df.columns if c not in ['life_expectancy', 'country_name', 'country_code']]

X_train = train_df[feature_cols]
y_train = train_df['life_expectancy']

print(f"Số lượng đặc trưng: {len(feature_cols)}")
print(f"Kích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")

In [ ]:
param_distributions_xgb = {
    'n_estimators': randint(200, 2000),
    'max_depth': randint(3, 12),
    'learning_rate': loguniform(1e-3, 3e-1),
    'subsample': uniform(0.6, 0.4),
    'colsample_bytree': uniform(0.5, 0.5),
    'gamma': loguniform(1e-3, 10),
    'reg_alpha': loguniform(1e-3, 10),
    'reg_lambda': loguniform(1e-3, 10),
}

xgb = XGBRegressor(objective='reg:squarederror', random_state=RANDOM_STATE, verbosity=0, n_jobs=-1)

random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_distributions_xgb,
    n_iter=60,
    scoring='neg_root_mean_squared_error',
    cv=5,
    n_jobs=-1,
    random_state=RANDOM_STATE,
    verbose=2
)

# Huấn luyện tìm kiếm siêu tham số
random_search.fit(X_train, y_train)

In [ ]:
best_xgb = random_search.best_estimator_
best_params = random_search.best_params_

print("="*60)
print("SIÊU THAM SỐ TỐI ƯU")
print("="*60)
for param, value in best_params.items():
    print(f"  {param:20s}: {value}")

In [ ]:
y_train_pred = best_xgb.predict(X_train)

train_mae = mean_absolute_error(y_train, y_train_pred)
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_r2 = r2_score(y_train, y_train_pred)

print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print("Model: XGBoost:")
print(f"RMSE loss: {train_rmse:.3f}")
print(f"MAE loss : {train_mae:.3f}")
print(f"R2 score : {train_r2:.3f}")

In [ ]:
os.makedirs('../model/7_xgboost', exist_ok=True)
model_path = '../model/7_xgboost/xgboost.pkl.gz'
joblib.dump(best_xgb, model_path, compress=('gzip', 9))
print(f"Saved best XGBoost model to: {model_path}")